In [1]:
# _*_ coding: utf-8 _*_
# @Time: 2021/12/09 16:05 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 南方医院丙戊酸：1. 数据处理

# 原始数据预处理

## 导入程序包

In [2]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()
print(project_path)

D:\PycharmProjects\VPA_southern_hospital


## 导入预定义函数

In [3]:
# 字符串转换为时间格式
import datetime
def str_to_datetime(x):
    try:
        a = datetime.datetime.strptime(x, "%d/%m/%Y %H:%M:%S")
        return a
    except:
        return np.NaN

In [4]:
# 过滤文字
import numpy as np
def filter_string(df,feature):
    # 过滤文字!!!!!!!!!!!!!!!!!!!!!!!!!!!
    df=df[df[feature].str.contains('\d')]
    return df

In [5]:
# 过滤特殊字符
def filter_spec_character(df,feature):
    df[feature]=df[feature].apply(lambda x: re.sub('<|>|\+|\/|-','',str(x)))
    return df

In [6]:
# 过滤异常大值
def filter_huge_value(df,feature,median_value=1):
    # 过滤异常大值!!!!!!!!!!!!!!!!!!!!!!!!!!
    if not median_value:
        median_value=df[feature].median()
    df[feature]=df[feature].apply(lambda x: np.nan if x<0 else x if float(x) <= 100 * median_value else 100*median_value)
    df=df[df[feature].notnull()]
    return df

In [7]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df,missing_list=[]):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    if not missing_list:
        for i in df.columns:
            if df[i].isnull().sum() > 0:
                missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)
        # 列出参数列表
        tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
        # 进行参数的搜索组合
        grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_grid_parameter,cv=3)
        #rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
        #根据已有数据去拟合随机森林模型
        grid.fit(x, y)
        rfr = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'])
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

## 连接数据库

In [8]:
# 丙戊酸-双相障碍1
# 连接数据库
conn = MySQLDB.connect(host='192.168.0.37', port=3306, user='root', password='111111', db='ndyy_vpa_sxza_1', charset='UTF8')
# conn = MySQLDB.connect(host='localhost', port=3306, user='root', password='123456', db='上海六院', charset='UTF8')
cursor = conn.cursor()
cursor.execute("Select version()")
for i in cursor:
    print(i)

('8.0.22',)


## 读取doctor_order数据库表

In [9]:
# 丙戊酸-双相障碍v1.0和丙戊酸-双相障碍v2.0
# 从df_doctor_order数据库读入数据
# 丙戊酸long_d_order不准，不要
try:
    sql = 'select patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc \
    from doctor_order \
    UNION \
    select patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc \
    from ndyy_vpa_sxza_2.doctor_order;'
    df_doctor_order = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [10]:
print(df_doctor_order.shape)
print(df_doctor_order['patient_id'].nunique())

(26568, 10)
218


In [11]:
# 保存doctor_order用药记录
df_doctor_order.to_excel(project_path+'/data/pre_processed_raw_data/df_doctor_order.xlsx')

## 读取inp_record住院记录

In [12]:
# 从inp_record表读入数据
try:
    sql = 'select patient_id,case_no,adm_date,care_area,dis_date,age from inp_record \
     UNION \
    select patient_id,case_no,adm_date,care_area,dis_date,age from ndyy_vpa_sxza_2.inp_record;'
    df_inp_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [13]:
print(df_inp_record.shape)
print(df_inp_record['patient_id'].nunique())

(241, 6)
218


In [14]:
#用来判断某列是否有缺失值
df_inp_record.isnull().any() 

patient_id    False
case_no       False
adm_date      False
care_area     False
dis_date       True
age           False
dtype: bool

In [15]:
df_inp_record[df_inp_record['dis_date'].isnull()]

,patient_id,case_no,adm_date,care_area,dis_date,age
0,6873870,10001302,2020-08-15 17:12:30,JSKXLHLDY-心理二区护理单元江高,NaT,28
2,6949939,10019847,2020-08-17 20:28:47,JSKXLHLDY-心理二区护理单元江高,NaT,27
4,369329,10028911,2020-08-18 15:12:28,JSKXLHLDY-心理二区护理单元江高,NaT,33
27,8043274,10914758,2020-11-18 17:38:49,JSKXLHLDY-心理二区护理单元江高,NaT,29


In [16]:
# 查看重复数据
df_inp_record[df_inp_record.duplicated(subset=['patient_id','adm_date'],keep='first')]

,patient_id,case_no,adm_date,care_area,dis_date,age


In [17]:
df_inp_record.to_excel(project_path+'/data/pre_processed_raw_data/df_inp_record.xlsx')

## 读取diagnostic_record

In [18]:
# 从medical_record表读入数据
try:
    sql = 'select patient_id,case_no,record_date,diagnostic_type,diagnostic_content from diagnostic_record \
        UNION \
        select patient_id,case_no,record_date,diagnostic_type,diagnostic_content from ndyy_vpa_sxza_2.diagnostic_record;'
    df_diagnostic_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [19]:
print(df_diagnostic_record.shape)
print(df_diagnostic_record['patient_id'].nunique())

(1873, 5)
359


In [20]:
#用来判断某列是否有缺失值
df_diagnostic_record.isnull().any() 

patient_id            False
case_no                True
record_date           False
diagnostic_type       False
diagnostic_content    False
dtype: bool

In [21]:
# 查看重复数据
df_diagnostic_record[df_diagnostic_record.duplicated(subset=['patient_id','case_no','record_date','diagnostic_content'],keep='first')]

,patient_id,case_no,record_date,diagnostic_type,diagnostic_content


In [22]:
writer=pd.ExcelWriter(project_path+'/data/pre_processed_raw_data/df_diagnostic_record.xlsx')
df_diagnostic_record.to_excel(writer)
writer.save()

## 读取test_result数据表

### test_record

In [23]:
# 从数据库读入test_record数据
try:
    sql = 'select test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time from test_record \
    UNION \
    select test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time from ndyy_vpa_sxza_2.test_record;'
    df_test_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [24]:
print(df_test_record.shape)
print(df_test_record['patient_id'].nunique())

(9953, 7)
364


In [25]:
df_test_record

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time
0,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
1,800001202207||B002||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
2,800001202207||B005||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
3,800001202207||B006||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
4,800001202207||B007||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35
...,...,...,...,...,...,...,...
9948,800014409316||B008||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:18
9949,800014409316||B010||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:47
9950,800014409316||B178||1,8004050,None,2021-12-24 11:55:00,1:双相情感障碍 2:锥体外系综合征,血清,2021-12-24 09:20:47
9951,800014409317||A083||1,8004050,None,2021-12-24 09:42:00,1:双相情感障碍 2:锥体外系综合征,静脉血,2021-12-24 09:20:18


In [26]:
#用来判断某列是否有缺失值
df_test_record.isnull().any() 

test_record_id        False
patient_id            False
case_no                True
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
dtype: bool

In [27]:
# 保存test_record
df_test_record.to_excel(project_path+'/data/pre_processed_raw_data/df_test_record.xlsx')

### test_result

In [28]:
# 从数据库读入test_result数据
try:
    sql = 'select test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym from test_result\
    UNION \
    select test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym from ndyy_vpa_sxza_2.test_result;'
    df_test_result = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [29]:
print(df_test_result.shape)

(61713, 7)


In [30]:
#用来判断某列是否有缺失值
df_test_result.isnull().any() 

test_record_id    False
project_name      False
test_result       False
result_unit        True
is_normal          True
refer_scope        True
synonym            True
dtype: bool

In [31]:
# 查看重复数据
df_test_result[df_test_result.duplicated(subset=['test_record_id','project_name','test_result'],keep='first')]

,test_record_id,project_name,test_result,result_unit,is_normal,refer_scope,synonym
59467,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgA_gd
59468,800013465956||B042||1,免疫球蛋白,阴性(-),None,N,阴性(-),IgM_gd


In [32]:
# 删除test_result数据
df_test_result=df_test_result.drop_duplicates(subset=['test_record_id','project_name','test_result'],keep='first')
df_test_result=df_test_result.reset_index(drop=True)

In [33]:
print(df_test_result.shape)

(61711, 7)


In [34]:
# 删除<>号
df_test_result=filter_spec_character(df_test_result,'test_result')

In [35]:
print(df_test_result.shape)

(61711, 7)


In [36]:
# 保存test_result
df_test_result.to_excel(project_path+'/data/pre_processed_raw_data/df_test_result.xlsx')

### test_record+test_result

In [37]:
df_test=pd.merge(df_test_record,df_test_result,on=['test_record_id'],how='inner')

In [38]:
df_test.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym
0,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,总二氧化碳,28.7,mmol/L,N,21.0-31.0,TCO2
1,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,尿酸,364,μmol/L,N,208-428,UA
2,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,肌酐,86,μmol/L,N,53-123,CR
3,800001202207||B001||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,尿素,4.7,mmol/L,N,2.8-7.2,UREA
4,800001202207||B002||1,7033812,None,2018-01-09 09:47:00,1:双相性情感障碍,血清,2018-01-09 08:05:35,天门冬氨酸氨基转移酶,25,U/L,N,15-40,AST


In [39]:
print(df_test.shape)
print(df_test['patient_id'].nunique())

(61711, 13)
364


In [40]:
# 保存检测数据
df_test.to_excel(project_path+'/data/pre_processed_raw_data/df_test.xlsx')

## 体检数据

In [41]:
# 从数据库读入体检exam数据
try:
    sql = 'select patientId,examDateTime,normalInd,description from exam_record \
        UNION select patientId,examDateTime,normalInd,description from ndyy_vpa_sxza_2.exam_record;'
    df_exam = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [42]:
# 删除空数据
df_exam=df_exam[df_exam['description'].notnull()].reset_index(drop=True)

In [43]:
# 重命名
df_exam=df_exam.rename(columns={'patientId':'patient_id',
                       'examDateTime':'exam_time'})

In [44]:
print(df_exam.shape)
print(df_exam['patient_id'].nunique())

(429, 4)
192


In [45]:
df_exam

,patient_id,exam_time,normalInd,description
0,8242965,2020-09-01 00:33:40,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
1,8242965,2020-09-01 00:33:40,阳性,双肾大小形态正常，轮廓清晰，包膜完整，实质回声均匀，集合系统回声正常，双侧输尿管不扩张。膀胱...
2,7006940,2020-09-01 00:33:40,阳性,主动脉扩张、迂曲、延长，主动脉结向左突出，见弧状钙化影；两侧肺野透光度正常，未见明确实变影；...
3,7006940,2020-09-01 00:33:40,阳性,肝脏形态大小正常，包膜完整光滑，肝内实质回声细密稍增强，肝内血管及胆管显示欠清晰，门静脉、胆...
4,8413999,2020-09-03 00:33:40,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
...,...,...,...,...
424,9080293,2021-10-22 10:32:30,阳性,头部平扫示双侧大脑半球对称，灰白质对比正常，脑实质内未见局灶性异常密度影。各脑室、脑...
425,7692024,2021-12-07 15:07:35,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理略增粗、增多...
426,7692024,2021-12-09 08:38:12,阴性,肝脏形态大小正常，包膜完整光滑，下缘角锐利，肝内实质回声均匀，肝内血管及胆管显示清晰，走行正...
427,8505473,2021-12-13 15:10:42,阴性,两侧胸廓对称、所见骨质未见异常；两侧肺野透过度正常，未见异常密度增高影；两肺纹理清晰，无增粗...


In [46]:
df_exam.to_excel(project_path+'/data/pre_processed_raw_data/df_exam.xlsx')

## 提取人口学特征

### gender、race

In [47]:
# 从patient_info提取gender,race,birth_year
try:
    sql = 'select patient_id,gender,race,birth_year from patient_info \
        UNION select patient_id,gender,race,birth_year from ndyy_vpa_sxza_2.patient_info;'
    df_patient_info = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [48]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(364, 4)
364


In [49]:
df_patient_info

,patient_id,gender,race,birth_year
0,1007763,女,汉族,1982-07-14
1,1039505,女,汉族,1996-05-13
2,110166,男,汉族,2003-06-03
3,1112533,女,汉族,1996-08-21
4,1126688,男,汉族,1990-09-28
...,...,...,...,...
359,7692024,女,汉族,2000-03-26
360,8505473,男,汉族,1996-06-03
361,9080293,男,汉族,1997-11-13
362,9424074,男,汉族,1994-09-26


In [50]:
#用来判断某列是否有缺失值
df_patient_info.isnull().any() 

patient_id    False
gender        False
race           True
birth_year    False
dtype: bool

In [51]:
# 查看重复数据
df_patient_info[df_patient_info.duplicated(subset=['patient_id','gender','race'],keep='first')]

,patient_id,gender,race,birth_year


In [52]:
print(df_patient_info.shape)
print(df_patient_info['patient_id'].nunique())

(364, 4)
364


In [53]:
df_patient_info.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_info.xlsx')

### age

In [54]:
df_age=df_inp_record[['patient_id','case_no','age']]

In [55]:
print(df_age.shape)
print(df_age['patient_id'].nunique())

(241, 3)
218


In [56]:
df_age

,patient_id,case_no,age
0,6873870,10001302,28
1,547307,10005574,25
2,6949939,10019847,27
3,7765586,10024628,16
4,369329,10028911,33
...,...,...,...
236,9080293,14400275,24
237,9424074,14701280,27
238,9442416,14712892,25
239,7692024,14900947,21


In [57]:
df_age.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_age.xlsx')

### height

In [58]:
# 从patient_sign_record提取height、weight
try:
    sql = 'select patient_id,case_no,record_date,sign_type,record_content,record_unit from patient_sign_record \
        UNION select patient_id,case_no,record_date,sign_type,record_content,record_unit from ndyy_vpa_sxza_2.patient_sign_record;'
    df_patient_sign_record = pd.read_sql(sql, conn)
except MySQLDB.err.ProgrammingError as e:
    print('surgical_record Error is ' + str(e))
    sys.exit()

In [59]:
df_patient_sign_record.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_patient_sign_record.xlsx')

In [60]:
# 提取身高信息
df_height=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('身高')]

In [61]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(252, 6)
217


In [62]:
df_height

,patient_id,case_no,record_date,sign_type,record_content,record_unit
0,6873870,10001302,2020-08-15 14:00:00,身高(cm),153,cm
79,547307,10005574,2020-08-16 14:00:00,身高(cm),162,cm
143,6949939,10019847,2020-08-17 14:00:00,身高(cm),160,cm
213,7765586,10024628,2020-08-18 14:00:00,身高(cm),168,cm
262,369329,10028911,2020-08-18 14:00:00,身高(cm),159,cm
...,...,...,...,...,...,...
19155,9080293,14400275,2021-10-20 14:00:00,身高(cm),175,cm
19249,9424074,14701280,2021-11-15 14:00:00,身高(cm),168,cm
19313,9442416,14712892,2021-11-16 14:00:00,身高(cm),171,cm
19383,7692024,14900947,2021-12-02 14:00:00,身高(cm),154,cm


In [63]:
# 过滤文字
df_height=filter_string(df_height,'record_content')

In [64]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(248, 6)
217


In [65]:
# 查看重复数据
df_height[df_height.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
6324,8881895,14103871,2021-10-13 14:00:00,身高(cm),168,cm
6425,8881895,14103871,2021-10-27 14:00:00,身高(cm),168,cm
6473,8881895,14103871,2021-11-03 14:00:00,身高(cm),168,cm
7508,7475493,4191397,2018-08-13 06:00:00,身高(cm),173,cm
7545,7475493,4191397,2018-08-20 14:00:00,身高(cm),173,cm
7786,6679951,4267891,2018-08-22 18:00:00,身高(cm),156,cm
7790,6679951,4267891,2018-08-22 14:00:00,身高(cm),156,cm
18043,8275190,9391429,2020-06-23 14:00:00,身高(cm),155,cm


In [66]:
# 排序
df_height=df_height.sort_values(by=['patient_id','case_no','record_date'],ascending=[True,True,True])
# 删除重复数据
df_height=df_height.drop_duplicates(subset=['patient_id','case_no','record_content'],keep='last')

In [67]:
print(df_height.shape)
print(df_height['patient_id'].nunique())

(240, 6)
217


In [68]:
df_height.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_height.xlsx')

### weight

In [69]:
# 提取体重信息
df_weight=df_patient_sign_record[df_patient_sign_record['sign_type'].str.contains('体重')]

In [70]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(820, 6)
218


In [71]:
df_weight

,patient_id,case_no,record_date,sign_type,record_content,record_unit
11,6873870,10001302,2020-08-15 14:00:00,体重(kg),52,kg
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
90,547307,10005574,2020-08-16 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
...,...,...,...,...,...,...
19370,9442416,14712892,2021-11-30 06:00:00,体重(kg),71,kg
19394,7692024,14900947,2021-12-02 14:00:00,体重(kg),85,kg
19413,7692024,14900947,2021-12-09 14:00:00,体重(kg),54,kg
19449,8505473,15026052,2021-12-11 14:00:00,体重(kg),46,kg


In [72]:
# 过滤文字
df_weight=filter_string(df_weight,'record_content')

In [73]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(815, 6)
218


In [74]:
# 过滤异常值
df_weight=df_weight[~ df_weight['record_content'].str.contains('0')]

In [75]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(688, 6)
190


In [76]:
# 查看重复数据
df_weight[df_weight.duplicated(subset=['patient_id','case_no','record_content'])]

,patient_id,case_no,record_date,sign_type,record_content,record_unit
30,6873870,10001302,2020-08-22 06:00:00,体重(kg),52,kg
60,6873870,10001302,2020-08-29 14:00:00,体重(kg),52,kg
111,547307,10005574,2020-08-23 06:00:00,体重(kg),52,kg
140,547307,10005574,2020-08-30 14:00:00,体重(kg),52,kg
177,6949939,10019847,2020-08-24 14:00:00,体重(kg),45,kg
...,...,...,...,...,...,...
19215,9080293,14400275,2021-11-03 14:00:00,体重(kg),57,kg
19238,9080293,14400275,2021-11-10 06:00:00,体重(kg),57,kg
19343,9442416,14712892,2021-11-23 14:00:00,体重(kg),71,kg
19370,9442416,14712892,2021-11-30 06:00:00,体重(kg),71,kg


In [77]:
# 排序
df_weight=df_weight.sort_values(by=['patient_id','case_no','record_date'],ascending=[True,True,True])
# 删除重复数据
df_weight=df_weight.drop_duplicates(subset=['patient_id','case_no','record_content'],keep='last')

In [78]:
print(df_weight.shape)
print(df_weight['patient_id'].nunique())

(246, 6)
190


In [79]:
df_weight.to_excel(project_path+'/data/pre_processed_raw_data/人口学特征/df_weight.xlsx')

# 纳排

## 纳入标准

### 提取丙戊酸用药

In [80]:
# 提取甲氨蝶呤的用药数据
# 注意药物缩写和药物音译名，wtf
df_VPA = df_doctor_order[df_doctor_order['drug_name'].str.contains('丙戊酸') & (~df_doctor_order['drug_name'].str.contains('药材科'))]

In [81]:
# 排序
df_VPA=df_VPA.sort_values(['patient_id','case_no','start_datetime'])

In [82]:
print(df_VPA.shape)
print(df_VPA['patient_id'].nunique())

(879, 10)
218


In [83]:
# 保存丙戊酸用药数据
df_VPA.to_excel(project_path+'/data/processed_data/df_2.1.1_VPA.xlsx')

### 双相情感障碍者

In [84]:
# 从diagnostic_record中提取双相情感障碍
df_BD=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('双相')]

In [85]:
df_BD['diagnostic_content'].unique()

array(['双相情感障碍，目前为伴有精神病性症状的躁狂发作', '双相情感障碍，目前为轻度或中度抑郁发作',
       '双相情感障碍目前为不伴精神病性症状躁狂发作', '双相情感障碍，目前为轻躁狂发作', '双相情感障碍，目前为中度抑郁发作',
       '双相情感障碍，目前为缓解状态', '双相情感障碍', '双相情感障碍，目前为混合性发作',
       '双相情感障碍目前为伴精神病的重度抑郁发作', '双相情感障碍目前为不伴精神病的重度抑郁发作', '非典型双相情感障碍',
       '双相情感障碍，目前为不伴有精神病性症状的重度抑郁发作', '双相情感障碍,快速循环型',
       '双相情感障碍，目前为躁狂发作，伴有精神病性症状', '双相情感障碍，目前为不伴有精神病性症状的躁狂发作', '双相性情感障碍',
       '双相情感障碍（恢复期）', '双相情感障碍,目前为中度抑郁发作', '双相情感障碍，伴有精神病性症状，目前为躁狂发作',
       '双相情感障碍不伴有精神病性症状的躁狂发作', '双相性情感障碍?', '双相情感障碍目前为伴精神病性症状的躁狂发作？',
       '双相情感障碍，目前为混合性发作？', '双相情感障碍，目前为重度抑郁发作', '双相情感障碍？',
       '双相情感障碍，目前为伴有精神病性症状的重度抑郁发作', '非典型双相情感障碍?',
       '双相情感障碍，目前为重度抑郁发作，伴有精神病性症'], dtype=object)

In [86]:
print(df_BD.shape)
print(df_BD['patient_id'].nunique())

(879, 5)
354


In [87]:
# 纳入双相情感障碍
df_VPA_BD=df_VPA[df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]

In [88]:
print(df_VPA_BD.shape)
print(df_VPA_BD['patient_id'].nunique())

(869, 10)
214


In [89]:
df_VPA_BD_not=df_VPA[~ df_VPA['patient_id'].isin(df_BD['patient_id'].unique())]
df_VPA_BD_not['patient_id'].unique()

array(['7506303', '7512212', '7528046', '7680010'], dtype=object)

In [90]:
# ### 年龄筛选：18-65岁

# df_age_adult=df_age[(df_age['age']>=18) & (df_age['age']<=65)]

# df_age_adult

# print(df_age_adult.shape)
# print(df_age_adult['patient_id'].nunique())

# # 纳入年龄限制
# df_VPA_age=df_VPA_BD[df_VPA_BD['patient_id'].isin(df_age_adult['patient_id'].unique())]

# print(df_VPA_age.shape)
# print(df_VPA_age['patient_id'].nunique())

# df_VPA_age.to_excel(project_path+'/data/processed_data/df_temp_VPA_age.xlsx')

### 达到血药浓度稳态

#### 计算丙戊酸日剂量

In [91]:
df_VPA_dosage=df_VPA_BD.copy()

# 清洗单次日剂量dosage，100ml--5g，1ml--0.05g
df_VPA_dosage['dosage']=df_VPA_dosage['dosage'].apply(lambda x: float(x.replace('ml', ''))*0.05 if 'ml' in x else x.replace('g', ''))

In [92]:
df_VPA_dosage.frequency.unique()

array(['1/日', '2/日', '1/早', '1/晚', 'ONCE', 'Tid', '1/午', '1/12小时',
       '3/日（餐前）', '1/日(餐前)'], dtype=object)

In [93]:
# 清洗频次
one=['1/午','ONCE','1/日','1/日(餐前)','1/早','1/晚']
two=['1/12小时','2/日']
three=['Tid','3/日（餐前）']
df_VPA_dosage['frequency']=df_VPA_dosage['frequency'].apply(lambda x: 1 if x in one else
                                                                2 if x in two else
                                                                3 if x in three else x)

In [94]:
# 计算日剂量
df_VPA_dosage['日剂量']=df_VPA_dosage['dosage'].astype('float') * df_VPA_dosage['frequency'].astype('float')

In [95]:
df_VPA_dosage

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
5389,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-03-29 19:14:29,NaT,核实,0.50
5396,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-03-31 09:09:50,NaT,核实,1.00
5312,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-07 09:10:18,NaT,核实,0.50
5313,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.25,1,2021-04-07 09:10:18,NaT,核实,0.25
5325,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-09 19:49:24,NaT,核实,0.50
...,...,...,...,...,...,...,...,...,...,...,...
26237,9424074,14701280,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-11-15 18:44:04,NaT,核实,1.00
26349,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-11-17 10:47:51,NaT,核实,0.50
26350,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-11-17 10:47:51,NaT,核实,0.50
26360,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-11-19 10:31:22,NaT,核实,1.00


In [96]:
print(df_VPA_dosage.shape)
print(df_VPA_dosage['patient_id'].nunique())
print(df_VPA_dosage['case_no'].nunique())

(869, 11)
214
237


In [97]:
df_VPA_dosage.to_excel(project_path+'/data/processed_data/df_2.1.4.1_计算日剂量.xlsx')

#### 提取用药日期

In [98]:
# 临时用药的结束时间=开始时间
aaa=df_VPA_dosage[df_VPA_dosage['end_datetime'].isnull()]
bbb=df_VPA_dosage[df_VPA_dosage['end_datetime'].notnull()]
aaa['end_datetime']=aaa['start_datetime']

In [99]:
aaa

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量
5389,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50
5396,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00
5312,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.50
5313,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.25,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.25
5325,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.50
...,...,...,...,...,...,...,...,...,...,...,...
26237,9424074,14701280,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-11-15 18:44:04,2021-11-15 18:44:04,核实,1.00
26349,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-11-17 10:47:51,2021-11-17 10:47:51,核实,0.50
26350,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-11-17 10:47:51,2021-11-17 10:47:51,核实,0.50
26360,9442416,14712892,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-11-19 10:31:22,2021-11-19 10:31:22,核实,1.00


In [100]:
df_VPA_time=pd.concat([aaa,bbb],axis=0)
df_VPA_time=df_VPA_time.reset_index(drop=True)
# 排序
df_VPA_time=df_VPA_time.sort_values(by=['patient_id','case_no','start_datetime'])

In [101]:
print(df_VPA_time.shape)
print(df_VPA_time['patient_id'].nunique())
print(df_VPA_time['case_no'].nunique())

(869, 11)
214
237


In [102]:
# 提取丙戊酸用药时间的年月日
df_VPA_time['start_date']=df_VPA_time['start_datetime'].apply(lambda x: str(x).split(' ')[0])
df_VPA_time['end_date']=df_VPA_time['end_datetime'].apply(lambda x: str(x).split(' ')[0])

In [103]:
df_VPA_time.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,0.50,2021-03-29,2021-03-29
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,1.00,2021-03-31,2021-03-31
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.50,2021-04-07,2021-04-07
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.25,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,0.25,2021-04-07,2021-04-07
4,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,0.50,2021-04-09,2021-04-09


In [104]:
df_VPA_time.to_excel(project_path+'/data/processed_data/df_2.1.4.2_提取年月日.xlsx')

#### 拆分长期医嘱和临时医嘱

In [105]:
# 同一剂量至少连续服用3天
# 打长期医嘱标签
df_VPA_time['long_d_order']=df_VPA_time['statusdesc'].apply(lambda x: 1 if re.match('停止',x) else 0)

In [106]:
# 长期医嘱和临时医嘱分开
drug_temp = df_VPA_time[df_VPA_time['long_d_order']==0]
drug_long = df_VPA_time[df_VPA_time['long_d_order']==1]

In [107]:
# 打临时时间标签，上午为0，下午为1
drug_temp['用药上下午标签']=drug_temp['start_datetime'].apply(lambda x: 0 if x.hour <13 else 1)

In [108]:
print(drug_temp.shape)
print(drug_temp['patient_id'].nunique())
print(drug_temp['case_no'].nunique())

(535, 15)
206
228


In [109]:
# ======= 单独对‘临时医嘱’处理 =======
drug_temp_all = drug_temp.drop_duplicates(subset=['patient_id','start_date'],keep='first')
drug_temp_1 = drug_temp.drop_duplicates(subset=['patient_id','start_date','用药上下午标签'],keep='first')
drug_temp_group = drug_temp_1.groupby(['patient_id','start_date'])['日剂量'].sum().reset_index()   # 同一天的临时医嘱相加，保证每天一条临时医嘱

# 拼接完整数据
drug_temp_result = pd.merge(drug_temp_all.drop(columns=['日剂量','用药上下午标签']),drug_temp_group,on=['patient_id','start_date'],how='inner')

In [110]:
print(drug_temp_result.shape)
print(drug_temp_result['patient_id'].nunique())
print(drug_temp_result['case_no'].nunique())

(463, 14)
206
228


In [111]:
drug_temp_result.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,start_date,end_date,long_d_order,日剂量
0,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-03-29 19:14:29,2021-03-29 19:14:29,核实,2021-03-29,2021-03-29,0,0.5
1,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-03-31 09:09:50,2021-03-31 09:09:50,核实,2021-03-31,2021-03-31,0,1.0
2,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-07 09:10:18,2021-04-07 09:10:18,核实,2021-04-07,2021-04-07,0,0.5
3,1112533,12210690,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2021-04-09 19:49:24,2021-04-09 19:49:24,核实,2021-04-09,2021-04-09,0,0.5
4,1173560,13397293,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2021-07-21 16:21:42,2021-07-21 16:21:42,核实,2021-07-21,2021-07-21,0,1.0


In [112]:
# 把‘临时医嘱’和‘长期医嘱’拼接
df_test_1 = pd.concat([drug_long,drug_temp_result],axis=0)
df_test_1 = df_test_1[drug_long.columns.tolist()]

# 拼接后重新排序
drug_vancomycin_4 = df_test_1.groupby(['case_no']).apply(
    lambda _df: _df.sort_values(by=['start_datetime'],ascending=1)).reset_index(drop=True)

In [113]:
drug_vancomycin_4.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-24 10:37:31,2020-09-04 11:38:00,停止,1.0,2020-08-24,2020-09-04,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,1,2020-08-25 17:31:34,2020-08-25 17:31:34,核实,0.5,2020-08-25,2020-08-25,0
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-09-04 11:38:55,2020-09-04 11:38:55,核实,1.0,2020-09-04,2020-09-04,0
4,547307,10005574,丙戊酸钠缓释片(德巴金)[0.5g*30片],1.0,0.5g*30片,0.5,2,2020-08-16 16:41:46,2020-08-31 11:15:00,停止,1.0,2020-08-16,2020-08-31,1


In [114]:
drug_vancomycin_4.to_excel(project_path+'/data/processed_data/df_2.1.4.3_拆分临时医嘱.xlsx')

#### 分裂日期

In [115]:
# 将时间拆开
def explode1(df, col):
    dftem1 = pd.DataFrame()
    for i in range(df.shape[0]):
        dftem = df.iloc[i].to_frame().T
        if isinstance(df.iloc[i][col], list):
            if len(df.iloc[i][col]) > 0:
                for j in range(len(df.iloc[i][col])):
                    dftem[col + "_"] = df.iloc[i][col][j]
                    dftem1 = pd.concat([dftem1, dftem])
            else:
                dftem1 = pd.concat([dftem1, dftem])
        else:
            dftem1 = pd.concat([dftem1, dftem])
    del dftem1[col]
    dftem1.rename(columns={col + "_": col}, inplace=True)
    return dftem1

In [116]:
# 生成 分裂日期
drug_vancomycin_4['分裂日期'] = ''
for i in range(len(drug_vancomycin_4)):
    drug_vancomycin_4['分裂日期'][i] = list(pd.date_range(start=drug_vancomycin_4['start_date'][i], 
                                                      end=drug_vancomycin_4['end_date'][i]))

##　＝＝　把时间List拆开　＝＝    
drug_vancomycin_5 = explode1(drug_vancomycin_4,'分裂日期')
drug_vancomycin_5['分裂日期']=drug_vancomycin_5['分裂日期'].apply(lambda x: str(x).split(' ')[0])

drug_vancomycin_5['起始标签'] = drug_vancomycin_5.apply(lambda x: '起始' if x['分裂日期']==x['start_date'] else 
                                                                '结尾' if x['分裂日期']==x['end_date'] else 
                                                                '期间', axis=1)

In [117]:
drug_vancomycin_5.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-15,起始
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间


In [118]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3759, 16)
214
237


In [119]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.4_分裂日期.xlsx')

#### 打时间标签

In [120]:
# 打日期标签
def flag_ttt(x):
    if (x['起始标签']!='结尾'):
        if x['start_datetime'].hour<13:
            return 0
        else:
            return 1
    else:
        if x['end_datetime'].hour<13:
            return 0
        else:
            return 1

In [121]:
# 上午是0，下午是1
# 起始和期间看 起始时间， 结尾看停止时间
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(lambda x: 0 if ((x['起始标签']!='结尾')and(x['start_datetime'].hour < 13)) else 1, axis=1)
drug_vancomycin_5['时间标签'] = drug_vancomycin_5.apply(flag_ttt,axis=1)

In [122]:
print(drug_vancomycin_5.shape)
print(drug_vancomycin_5['patient_id'].nunique())
print(drug_vancomycin_5['case_no'].nunique())

(3759, 17)
214
237


In [123]:
drug_vancomycin_5.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-15,起始,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间,1
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间,1


In [124]:
drug_vancomycin_5.to_excel(project_path+'/data/processed_data/df_2.1.4.5_打时间标签.xlsx')

#### 调整日剂量

In [125]:
temp_l = []

for i in drug_vancomycin_5.case_no.unique():
    temp = drug_vancomycin_5[drug_vancomycin_5['case_no']==i].reset_index(drop=True)
    
#     a = 1
#     print('===={}===='.format(i))
    for j in temp.分裂日期.unique():
#         print(a,j)
#         a+=1
        temp_2 = temp[temp['分裂日期']==j]
        if 0 in temp_2['long_d_order'].unique():
            temp_long = temp_2[temp_2['long_d_order']==1].tail(1).reset_index(drop=True)
            temp_st = temp_2[temp_2['long_d_order']==0]
            temp_st['日剂量'] = temp_st['日剂量'].sum()
            temp_st = temp_st.tail(1).reset_index(drop=True)
#             print(temp_long.shape[0],temp_st.shape[0])
            # 同一天，既有长期医嘱又有短期医嘱
            if len(temp_long)>0:
#                 print('long>0')
                flag = temp_long.起始标签.unique()[0]
                if flag == '期间':
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] + temp_st.loc[0,'日剂量']                           
                    temp_l.append(temp_long)
                else:
                    temp_l.append(temp_st)
            # 该天全是短期医嘱
            else:
#                 print(temp_st)
#                 print('***')
                temp_l.append(temp_st)
        # 该天全是长期医嘱
        else:
            temp_long = temp_2.tail(1).reset_index(drop=True)
            flag_l = temp_2.起始标签.unique().tolist()
            flag_time_l = temp_2.时间标签.unique().tolist()
            if ('结尾' in flag_l and '起始' in flag_l) or len(flag_time_l)>1:
                # 按剂量去重（防止上一次结束，与下一次开始是两种情况）
                temp_long.loc[0,'日剂量'] = temp_2.drop_duplicates(subset=['日剂量'],keep='last').日剂量.mean()
                temp_l.append(temp_long)
            else:
                flag = temp_long.loc[0,'起始标签']
                flag_time = temp_long.loc[0,'时间标签']
                # 开头：大于13点取半天，  结尾：小于13点取半天
                if flag=='起始' and flag_time==1:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2
                if flag=='结尾' and flag_time==0:
                    temp_long.loc[0,'日剂量'] = temp_long.loc[0,'日剂量'] / 2

                temp_l.append(temp_long)
        
df_VPA_modification = pd.concat(temp_l,axis=0).reset_index(drop=True)
df_VPA_modification['日剂量']=df_VPA_modification['日剂量'].apply(lambda x: round(x,2))
del temp_l

In [126]:
print(df_VPA_modification.shape)
print(df_VPA_modification['patient_id'].nunique())
print(df_VPA_modification['case_no'].nunique())

(3189, 17)
214
237


In [127]:
df_VPA_modification.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.25,2020-08-15,2020-08-24,1,2020-08-15,起始,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-16,期间,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-17,期间,1
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-18,期间,1
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.50,2020-08-15,2020-08-24,1,2020-08-19,期间,1


In [128]:
df_VPA_modification.to_excel(project_path+'/data/processed_data/df_2.1.4.6_调整日剂量.xlsx')

#### 用药稳态筛选

In [129]:
df_VPA_steady=df_VPA_modification.copy()
# 到稳态血药浓度（同一剂量至少连续服用3天)
temp_list=[]
for i in df_VPA_steady['case_no'].unique():
    print(i)
    temp=df_VPA_steady[df_VPA_steady['case_no']==i]
    temp=temp.reset_index(drop=True)
    j=0
    while j < temp.shape[0]:
#         print(j)
        # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
        dosage_judge=temp.loc[j,'日剂量']    
        # 1. 剂量判断
        break_bool=False
        for k in range(j+1,temp.shape[0]):
            differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
            if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
                # 判断是否中断
                break_bool=True
                break
        # 是否是最后一天
        if k != temp.shape[0]-1:
            # 是否是稳态
            if k-1-j>=2:
                temp_list.append(temp.loc[j:k-1,])
            j=k
            continue
        else:
            # 最后一天是否中断
            if break_bool:
                # 是否是稳态
                if k-1-j>=2:
                    temp_list.append(temp.loc[j:k-1,])
            else:
                if k-j >=2:
                    temp_list.append(temp.loc[j:k,])
            j=k+1

10001302
10005574
10019847
10024628
10028911
10130473
10146112
10171606
10200584
10293806
10297961
10338392
10343367
10357709
10368285
10389652
10526969
10559425
10582637
10588084
10616235
10674663
10692337
10830666
10859517
10864711
10877290
10914758
10941636
10989230
11022153
11069250
11103754
11200975
11223919
11238946
11291598
11351405
11547061
11747590
11787937
12001085
12055514
12069121
12070485
12165203
12210690
12238352
12297734
12313528
12325442
12327058
12415154
12550325
12610841
12631919
12693940
12696147
12782405
12835527
12855725
13022476
13063464
13150462
13176502
13266036
13317922
13335769
13397293
13425571
13491055
13561501
13692578
13817948
13951019
13997754
14040566
14103871
14107745
14120909
14135942
14184132
14322457
14400275
14701280
14712892
14900947
15026052
4028318
4045163
4069083
4103620
4112018
4133350
4151975
4191397
4216082
4236927
4255479
4267891
4283749
4304073
4364618
4366097
4446056
4446680
4536465
4553217
4560293
4569640
4579522
4592169
4707313
4761372


In [130]:
# list转换为DataFrame
df_VPA_steady = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [131]:
print(df_VPA_steady.shape)
print(df_VPA_steady['patient_id'].nunique())
print(df_VPA_steady['case_no'].nunique())

(2314, 17)
146
157


In [132]:
df_VPA_steady.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,日剂量,start_date,end_date,long_d_order,分裂日期,起始标签,时间标签
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-16,期间,1
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-17,期间,1
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-18,期间,1
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-19,期间,1
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,0.5,2020-08-15,2020-08-24,1,2020-08-20,期间,1


In [133]:
df_VPA_steady.to_excel(project_path+'/data/processed_data/df_2.1.4.7_用药稳态筛选.xlsx')

In [134]:
# ### 无躯体疾病

# # 无严重的或不稳定的躯体疾病，且经检查表明躯体健康（包括体检查、病史、心电图和血生化、血液学及尿检查）

# #### 无体检健康异常

# df_VPA_exam=df_VPA_steady.copy()
# # 删除体检异常
# for i in range(df_exam.shape[0]):
#     description=df_exam.loc[i,'description']
#     description_list=re.split('，|。',description)
#     for k in description_list:
#         if re.match('[^无否未见]*异常',k):
#             print(i)
#             df_VPA_exam=df_VPA_exam[df_VPA_exam['patient_id']!=df_exam.loc[i,'patient_id']]
#             break

# print(df_VPA_exam.shape)
# print(df_VPA_exam['patient_id'].nunique())
# print(df_VPA_exam['case_no'].nunique())

# #### 病史

# df_diagnostic_record.columns

## 排除标准

### 妊娠或哺乳期妇女

In [135]:
# 字段“尿妊娠试验”阳性为怀孕；“β人绒毛膜促性腺激素”在正常值范围内表明没有怀孕；
df_pregnant_1=df_test[df_test['project_name'].str.contains('尿妊娠试验')]
df_pregnant_1=df_pregnant_1[df_pregnant_1['test_result'].str.contains('阳性')]
df_pregnant_2=df_test[df_test['project_name'].str.contains('β人绒毛膜促性腺激素')]
df_pregnant_2=df_pregnant_2[df_pregnant_2['test_result'].astype('float')>5.3]

In [136]:
df_test_pregnant=pd.concat([df_pregnant_1,df_pregnant_2],axis=0)

In [137]:
df_test_pregnant

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym
19330,800005121742||C122||1,7565842,6917667,2019-07-13 11:34:00,双相情感障碍，目前为重度抑郁发作,血清,NaT,β人绒毛膜促性腺激素,107584.000,mIU/mL,N,绝经后(0.00-8.30) 绝经前(0.00-5.30)\n\r,βHCG


In [138]:
# 剔除妊娠妇女
df_VPA_pregnant=df_VPA_steady[~ df_VPA_steady['patient_id'].isin(df_test_pregnant.patient_id.unique())]

In [139]:
print(df_VPA_pregnant.shape)
print(df_VPA_pregnant['patient_id'].nunique())
print(df_VPA_pregnant['case_no'].nunique())

(2302, 17)
145
156


In [140]:
# 排除妊娠妇女
df_VPA_pregnant.to_excel(project_path+'/data/processed_data/df_2.2.1_排除妊娠妇女.xlsx')

### 排除数据严重缺失

In [141]:
# 数据严重缺失（如患者ID、主要研究结局等
df_VPA_pregnant.isnull().any()

patient_id        False
case_no           False
drug_name         False
amount            False
drug_spec         False
dosage            False
frequency         False
start_datetime    False
end_datetime      False
statusdesc        False
日剂量               False
start_date        False
end_date          False
long_d_order      False
分裂日期              False
起始标签              False
时间标签              False
dtype: bool

# 合并人口学特征

## 合并人口学信息

In [142]:
# 取 patient_id和case_no
df_popu=df_VPA_pregnant[['patient_id','case_no']]

In [143]:
# merge gender
df_popu=pd.merge(df_popu,df_patient_info,on=['patient_id'],how='left')

In [144]:
# merge age
df_age_simple=df_age[['case_no','age']]
df_popu=pd.merge(df_popu,df_age_simple,on=['case_no'],how='left')

In [145]:
# merge height
df_height_simple=df_height[['case_no','record_content']]
df_height_simple=df_height_simple.rename(columns={'record_content':'身高'})
df_popu=pd.merge(df_popu,df_height_simple,on=['case_no'],how='left')

In [146]:
# merge weight
df_weight_simple=df_weight[['case_no','record_content']]
df_weight_simple=df_weight_simple.rename(columns={'record_content':'体重'})
df_popu=pd.merge(df_popu,df_weight_simple,on=['case_no'],how='left')

In [147]:
df_popu.head()

,patient_id,case_no,gender,race,birth_year,age,身高,体重
0,6873870,10001302,女,汉族,1992-11-06,28,153,52
1,6873870,10001302,女,汉族,1992-11-06,28,153,52
2,6873870,10001302,女,汉族,1992-11-06,28,153,52
3,6873870,10001302,女,汉族,1992-11-06,28,153,52
4,6873870,10001302,女,汉族,1992-11-06,28,153,52


In [148]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(3082, 8)
145


In [149]:
# 删除重复数据
df_popu=df_popu.drop_duplicates(['patient_id','case_no','gender'])

In [150]:
print(df_popu.shape)
print(df_popu['patient_id'].nunique())

(156, 8)
145


In [151]:
df_popu.to_excel(project_path+'/data/processed_data/df_3.1_popu.xlsx')

## 计算BMI

In [152]:
BMI_list=[round(x*10000/y/y,2) for (x,y) in zip(df_popu['体重'].astype('float').tolist(),df_popu['身高'].astype('float').tolist())]

In [153]:
print(len(BMI_list))
print(df_popu.shape)

156
(156, 8)


In [154]:
df_popu['BMI']=np.array(BMI_list)

In [155]:
df_popu

,patient_id,case_no,gender,race,birth_year,age,身高,体重,BMI
0,6873870,10001302,女,汉族,1992-11-06,28,153,52,22.21
18,547307,10005574,女,汉族,1995-05-13,25,162,52,19.81
32,6949939,10019847,女,汉族,1993-11-22,27,160,45,17.58
47,7765586,10024628,女,汉族,2003-12-08,16,168,82,29.05
57,369329,10028911,女,汉族,1987-11-03,33,159,67,26.50
...,...,...,...,...,...,...,...,...,...
3045,2583650,9818116,女,汉族,1961-10-27,59,158,76,30.44
3049,7411504,9835190,女,汉族,2003-10-15,17,163,57,21.45
3054,8334957,9890262,男,汉族,2006-07-25,14,166,49,17.78
3068,6865481,9910592,男,汉族,2000-11-06,20,183,NaN,NaN


## 并入人口学特征

In [156]:
df_popu=df_popu[['patient_id','gender','age','身高','体重','BMI']]

In [157]:
df_VPA_popu=pd.merge(df_VPA_pregnant,df_popu,on=['patient_id'],how='left')

In [158]:
print(df_VPA_popu.shape)
print(df_VPA_popu['patient_id'].nunique())
print(df_VPA_popu['case_no'].nunique())

(2610, 22)
145
156


In [159]:
# 合并人口学特征
df_VPA_popu.to_excel(project_path+'/data/processed_data/df_3.3_并入人口学特征.xlsx')

# 合并个人史、既往史

## 合并既往史

In [160]:
# 高血压、糖尿病
df_VPA_history=df_VPA_popu.copy()
# 提取高血压
df_EH=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('高血压')]
df_VPA_history['高血压']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_EH.patient_id.unique() else 0)
# 提取糖尿病
df_DM=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('糖尿病')]
df_VPA_history['糖尿病']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_DM.patient_id.unique() else 0)

In [161]:
df_EH.patient_id.unique()

array(['2930481', '6591495', '2583650', '7836330'], dtype=object)

In [162]:
df_DM.patient_id.unique()

array(['7006940', '7651365', '8881895', '5878393', '2583650'],
      dtype=object)

In [163]:
print(df_VPA_history.shape)
print(df_VPA_history['patient_id'].nunique())
print(df_VPA_history['case_no'].nunique())

(2610, 24)
145
156


In [164]:
df_VPA_history.to_excel(project_path+'/data/processed_data/df_4.1_合并既往史.xlsx')

## 合并脑器质疾病

In [165]:
# 脑器质疾病：脑退行性改变（痴呆）、脑血管病（脑梗塞/缺血性卒中/中风、脑出血）、颅内感染、脑外伤、脑肿瘤、癫痫

In [166]:
# 提取脑器质疾病brain organic mental disorders
df_bomd=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('器质'))&
                            df_diagnostic_record['diagnostic_content'].str.contains('精神')]

In [167]:
df_VPA_history['脑器质疾病']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_bomd.patient_id.unique() else 0)

In [168]:
df_VPA_history.to_excel(project_path+'/data/processed_data/df_4.2_合并脑器质疾病.xlsx')

## 合并肝肾疾病

In [169]:
# 提取肝炎
df_HCA=df_diagnostic_record[df_diagnostic_record['diagnostic_content'].str.contains('肝炎')]
df_VPA_history['肝炎']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_HCA.patient_id.unique() else 0)

In [170]:
# 提取肾病，不包括肾结石
df_NEP=df_diagnostic_record[(df_diagnostic_record['diagnostic_content'].str.contains('肾'))&
                           (~ df_diagnostic_record['diagnostic_content'].str.contains('肾结石'))]
df_VPA_history['肾病']=df_VPA_history['patient_id'].apply(lambda x: 1 if x in df_NEP.patient_id.unique() else 0)

In [171]:
print(df_VPA_history.shape)
print(df_VPA_history['patient_id'].nunique())
print(df_VPA_history['case_no'].nunique())

(2610, 27)
145
156


In [172]:
df_VPA_history.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,gender,age,身高,体重,BMI,高血压,糖尿病,脑器质疾病,肝炎,肾病
0,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,女,28,153,52,22.21,0,0,0,0,0
1,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,女,28,153,52,22.21,0,0,0,0,0
2,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,女,28,153,52,22.21,0,0,0,0,0
3,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,女,28,153,52,22.21,0,0,0,0,0
4,6873870,10001302,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,1,2020-08-15 19:51:46,2020-08-24 10:37:37,停止,...,女,28,153,52,22.21,0,0,0,0,0


In [173]:
# 保存纳入标准数据
df_VPA_history.to_excel(project_path+'/data/processed_data/df_4.3_合并肝肾疾病.xlsx')

# 并入出入院时间

In [174]:
df_inp_record_simple=df_inp_record[['case_no','adm_date','dis_date']]

In [175]:
# 并入出入院时间
df_VPA_inp_time=pd.merge(df_VPA_history,df_inp_record_simple,on='case_no',how='left')

In [176]:
print(df_VPA_inp_time.shape)
print(df_VPA_inp_time['patient_id'].nunique())
print(df_VPA_inp_time['case_no'].nunique())

(2610, 29)
145
156


## 补充住院时间

In [177]:
# 补充住院时间
aaa=df_VPA_inp_time[df_VPA_inp_time['adm_date'].isnull()]
bbb=df_VPA_inp_time[df_VPA_inp_time['adm_date'].notnull()]
aaa['adm_date']=aaa['start_datetime']
df_VPA_inp_time=pd.concat([aaa,bbb],axis=0)
df_VPA_inp_time=df_VPA_inp_time.sort_values(by=['patient_id','case_no','分裂日期'])
df_VPA_inp_time=df_VPA_inp_time.reset_index(drop=True)

In [178]:
# 补充出院时间
aaa=df_VPA_inp_time[df_VPA_inp_time['dis_date'].isnull()]
bbb=df_VPA_inp_time[df_VPA_inp_time['dis_date'].notnull()]
aaa['dis_date']=aaa['end_datetime']
df_VPA_inp_time=pd.concat([aaa,bbb],axis=0)
df_VPA_inp_time=df_VPA_inp_time.sort_values(by=['patient_id','case_no','分裂日期'])
df_VPA_inp_time=df_VPA_inp_time.reset_index(drop=True)

In [179]:
df_VPA_inp_time.to_excel(project_path+'/data/processed_data/df_5.1_补充住院时间.xlsx')

## 计算住院时长

In [180]:
# 计算住院时长
df_VPA_inp_time['住院时长']=(pd.to_datetime(df_VPA_inp_time['dis_date']) - pd.to_datetime(df_VPA_inp_time['adm_date']))
df_VPA_inp_time['住院时长']=df_VPA_inp_time['住院时长'].apply(lambda x:x.days+1)

In [181]:
print(df_VPA_inp_time.shape)
print(df_VPA_inp_time['patient_id'].nunique())
print(df_VPA_inp_time['case_no'].nunique())

(2610, 30)
145
156


In [182]:
df_VPA_inp_time.to_excel(project_path+'/data/processed_data/df_5.2_计算住院时长.xlsx')

# 并入安全性指标

## 提取安全性指标

In [183]:
# 肝功能异常：丙氨酸氨基转移酶(0-40U/L)--24、天门冬氨酸氨基转移酶(0-35U/L)、转氨酶比值(1.0-1.5)、总蛋白(65.0-85.0)、白蛋白(40.0-55.0)、
        # 球蛋白(20.0-40.0)、白/球比值(1.2-2.4)（有一个指标不正常就算肝功能异常）
# 皮疹：复方酮氯乳膏、莫米松乳膏（在治疗后使用，算为不良反应）
# 恶心：多潘立酮、甲氧氯普胺（在治疗后使用，算为不良反应）
# 血小板减少：血小板计数<100x109（在治疗后）

### 肝功能异常

#### 丙氨酸氨基转移酶ALT

In [184]:
df_test_ALT=df_test[df_test['project_name'].str.contains('丙氨酸氨基转移酶')]

# 患者限定
df_test_ALT=df_test_ALT[df_test_ALT.case_no.isin(df_VPA_inp_time.case_no.unique())]

# 指标0-1转换
df_test_ALT['肝功_label']=df_test_ALT['test_result'].apply(lambda x: 0 if float(x)<=40 else 1)

In [185]:
df_test_ALT.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2345,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,丙氨酸氨基转移酶,17,U/L,N,9-50,ALT,0
2532,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,丙氨酸氨基转移酶,42,U/L,H,7-40,ALT,1
2665,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,丙氨酸氨基转移酶,24,U/L,N,7-40,ALT,0
2794,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,丙氨酸氨基转移酶,9,U/L,N,9-50,ALT,0
2901,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,丙氨酸氨基转移酶,7,U/L,L,9-50,ALT,0


#### 天门冬氨酸氨基转移酶AST

In [186]:
df_test_AST=df_test[df_test['project_name'].str.contains('天门冬氨酸氨基转移酶')]

# 患者限定
df_test_AST=df_test_AST[df_test_AST.case_no.isin(df_VPA_inp_time.case_no.unique())]

# 指标0-1转换
df_test_AST['肝功_label']=df_test_AST['test_result'].apply(lambda x: 0 if float(x)<=35 else 1)

In [187]:
df_test_AST.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2340,800001934594||B002||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,天门冬氨酸氨基转移酶,18,U/L,N,15-40,AST,0
2346,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,天门冬氨酸氨基转移酶,18,U/L,N,15-40,AST,0
2527,800001953715||B002||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,天门冬氨酸氨基转移酶,30,U/L,N,13-35,AST,0
2533,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,天门冬氨酸氨基转移酶,30,U/L,N,13-35,AST,0
2660,800001979565||B002||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,天门冬氨酸氨基转移酶,32,U/L,N,13-35,AST,0


#### 转氨酶比值

In [188]:
df_test_zhuananmei=df_test[df_test['project_name'].str.contains('转氨酶比值')]
# 患者限定
df_test_zhuananmei=df_test_zhuananmei[df_test_zhuananmei.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_zhuananmei['肝功_label']=df_test_zhuananmei['test_result'].apply(lambda x: 0 if 1.0<=float(x)<=1.5 else 1)

In [189]:
df_test_zhuananmei.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2347,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,转氨酶比值,1.1,None,N,1.0-1.5,AST/ALT,0
2534,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,转氨酶比值,0.7,None,L,1.0-1.5,AST/ALT,1
2667,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,转氨酶比值,1.3,None,N,1.0-1.5,AST/ALT,0
2796,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,转氨酶比值,1.6,None,H,1.0-1.5,AST/ALT,1
2903,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,转氨酶比值,1.4,None,N,1.0-1.5,AST/ALT,0


#### 总蛋白TP

In [190]:
df_test_TP=df_test[df_test['project_name']=='总蛋白']
# 患者限定
df_test_TP=df_test_TP[df_test_TP.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_TP['肝功_label']=df_test_TP['test_result'].apply(lambda x: 0 if 65<=float(x)<=85 else 1)

In [191]:
df_test_TP.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2348,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,总蛋白,66.3,g/L,N,65.0-85.0,TP,0
2535,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,总蛋白,62.1,g/L,L,65.0-85.0,TP,1
2668,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,总蛋白,76.2,g/L,N,65.0-85.0,TP,0
2797,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,总蛋白,78.9,g/L,N,65.0-85.0,TP,0
2904,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,总蛋白,73.8,g/L,N,65.0-85.0,TP,0


#### 白蛋白ALB

In [192]:
df_test_ALB=df_test[df_test['project_name']=='白蛋白']
# 患者限定
df_test_ALB=df_test_ALB[df_test_ALB.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_ALB['肝功_label']=df_test_ALB['test_result'].apply(lambda x: 0 if 40<=float(x)<=55 else 1)

In [193]:
df_test_ALB.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2349,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,白蛋白,41.4,g/L,N,40.0-55.0,ALB,0
2536,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,白蛋白,38.9,g/L,L,40.0-55.0,ALB,1
2669,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,白蛋白,42.4,g/L,N,40.0-55.0,ALB,0
2798,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,白蛋白,47.6,g/L,N,40.0-55.0,ALB,0
2905,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,白蛋白,45.6,g/L,N,40.0-55.0,ALB,0


#### 球蛋白lgG

In [194]:
df_test_lgG=df_test[df_test['project_name']=='球蛋白']
# 患者限定
df_test_lgG=df_test_lgG[df_test_lgG.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_lgG['肝功_label']=df_test_lgG['test_result'].apply(lambda x: 0 if 20<=float(x)<=40 else 1)

In [195]:
df_test_lgG.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2350,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,球蛋白,24.9,g/L,N,20.0-40.0,G,0
2537,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,球蛋白,23.2,g/L,N,20.0-40.0,G,0
2670,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,球蛋白,33.8,g/L,N,20.0-40.0,G,0
2799,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,球蛋白,31.3,g/L,N,20.0-40.0,G,0
2906,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,球蛋白,28.2,g/L,N,20.0-40.0,G,0


#### 白球比值AG

In [196]:
df_test_AG=df_test[df_test['project_name'].str.contains('白/球比值')]
# 患者限定
df_test_AG=df_test_AG[df_test_AG.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_AG['肝功_label']=df_test_AG['test_result'].apply(lambda x: 0 if 1.2<=float(x)<=2.4 else 1)

In [197]:
df_test_AG.head()

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,肝功_label
2351,800001934594||B005||1,365567,4028318,2018-07-18 13:31:00,双相情感障碍，目前为混合性发作,血清,2018-07-18 07:25:07,白/球比值,1.7,None,N,1.2-2.4,A/G,0
2538,800001953715||B005||1,1819998,4045163,2018-07-20 11:02:00,双相情感障碍，目前为混合性发作,血清,2018-07-20 07:40:33,白/球比值,1.7,None,N,1.2-2.4,A/G,0
2671,800001979565||B005||1,7412218,4069083,2018-07-24 11:48:00,双相情感障碍，目前为伴有精神病性症状的躁狂发作,血清,2018-07-24 07:05:36,白/球比值,1.3,None,N,1.2-2.4,A/G,0
2800,800002012932||B005||1,7459164,4103620,2018-07-27 11:22:00,双相情感障碍，目前为混合性发作,血清,2018-07-27 07:40:37,白/球比值,1.5,None,N,1.2-2.4,A/G,0
2907,800002023150||B005||1,7461862,4112018,2018-07-28 11:40:00,双相情感障碍目前为不伴精神病的重度抑郁发作,血清,2018-07-28 07:38:51,白/球比值,1.6,None,N,1.2-2.4,A/G,0


#### 合并肝功能指标

In [198]:
df_test_gangong=pd.concat([df_test_ALT,df_test_AST],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_zhuananmei],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_TP],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_ALB],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_lgG],axis=0)
df_test_gangong=pd.concat([df_test_gangong,df_test_AG],axis=0)

In [199]:
df_test_gangong.project_name.unique()

array(['丙氨酸氨基转移酶', '天门冬氨酸氨基转移酶', '转氨酶比值', '总蛋白', '白蛋白', '球蛋白', '白/球比值'],
      dtype=object)

In [200]:
df_test_gangong.to_excel(project_path+'/data/processed_data/df_6.1.1_肝功能指标.xlsx')

### 皮疹

In [201]:
df_test_pizhen=df_doctor_order[df_doctor_order.drug_name.str.contains('复方酮氯乳膏|莫米松乳膏')]
# 患者限定
df_test_pizhen=df_test_pizhen[df_test_pizhen.case_no.isin(df_VPA_inp_time.case_no.unique())]

In [202]:
df_test_pizhen.drug_name.unique()

array(['莫米松乳膏[5g*1支]', '复方酮氯乳膏(自产)[20g*1支]'], dtype=object)

In [203]:
df_test_pizhen.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
16228,6637533,6085020,莫米松乳膏[5g*1支],1.0,5g*1支,5g,1/日,2019-04-19 10:12:04,NaT,核实
16809,5264695,6338386,莫米松乳膏[5g*1支],1.0,5g*1支,5g,1/日,2019-05-22 16:45:25,NaT,核实
17287,7816403,6494466,莫米松乳膏[5g*1支],1.0,5g*1支,5g,ONCE,2019-06-06 10:41:33,NaT,核实
19142,7946082,7284844,复方酮氯乳膏(自产)[20g*1支],1.0,20g*1支,20g,ONCE,2019-11-03 11:17:56,NaT,核实


In [204]:
df_test_pizhen.to_excel(project_path+'/data/processed_data/df_6.1.2_皮疹指标.xlsx')

### 恶心

In [205]:
df_test_exin=df_doctor_order[df_doctor_order.drug_name.str.contains('多潘立酮|甲氧氯普胺')]
# 患者限定
df_test_exin=df_test_exin[df_test_exin.case_no.isin(df_VPA_inp_time.case_no.unique())]

In [206]:
df_test_exin.drug_name.unique()

array(['多潘立酮片(吗丁啉)g*42片]', '甲氧氯普胺注射液(胃复安)l/10mg*1支]'], dtype=object)

In [207]:
df_test_exin.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
10687,7461862,4112018,多潘立酮片(吗丁啉)[10mg*42片],1.0,10mg*42片,10mg,ONCE,2018-08-10 14:51:54,NaT,核实
10691,7461862,4112018,多潘立酮片(吗丁啉)[10mg*42片],1.0,10mg*42片,10mg,2/日(餐前),2018-08-11 11:02:10,2018-08-14 08:12:00,停止
10699,7461862,4112018,多潘立酮片(吗丁啉)[10mg*42片],1.0,10mg*42片,10mg,2/日(餐前),2018-08-13 09:52:19,NaT,核实
11205,7481186,4236927,多潘立酮片(吗丁啉)[10mg*42片],1.0,10mg*42片,10mg,ONCE,2018-08-18 16:34:52,NaT,核实
13814,6019394,5125578,多潘立酮片(吗丁啉)[10mg*42片],1.0,10mg*42片,20mg,ONCE,2018-12-16 10:00:45,NaT,核实


In [208]:
df_test_exin.to_excel(project_path+'/data/processed_data/df_6.1.3_恶心指标.xlsx')

### 血小板计数<100

In [209]:
df_test_PLT=df_test[df_test['project_name'].str.contains('血小板计数')]
# 患者限定
df_test_PLT=df_test_PLT[df_test_PLT.case_no.isin(df_VPA_inp_time.case_no.unique())]
# 指标0-1转换
df_test_PLT['PLT_label']=df_test_PLT['test_result'].apply(lambda x: 1 if float(x)<=100 else 0)

In [210]:
df_test_PLT.project_name.unique()

array(['血小板计数'], dtype=object)

In [211]:
df_test_PLT.to_excel(project_path+'/data/processed_data/df_6.1.2_血小板计数指标.xlsx')

## 安全性指标转置

In [212]:
# # 扩展新列
# gangong_name_list=['丙氨酸氨基转移酶', '天门冬氨酸氨基转移酶', '转氨酶比值', '总蛋白', '白蛋白', '球蛋白', '白/球比值']
# pizhen_name_list=['皮疹']
# exin_name_list=['恶心']
# PLT_name_list=['血小板计数']

# indicators_name_list=gangong_name_list+pizhen_name_list+exin_name_list+PLT_name_list
# days_list=['_before','_after']

# for i in indicators_name_list:
#     for j in days_list:
#         df_VPA_indicators.loc[:,i+j]=np.nan

In [213]:
# # 安全指标数据
# gangong_data_list=[df_test_ALT,df_test_AST,df_test_zhuananmei,df_test_TP,df_test_ALB,df_test_lgG,df_test_AG]
# pizhen_data_list=[df_test_pizhen]
# exin_data_list=[df_test_exin]
# PLT_data_list=[df_test_PLT]

In [214]:
df_VPA_indicators=df_VPA_inp_time.copy()
df_VPA_indicators=df_VPA_indicators.reset_index(drop=True)
# 将一次用药的不同检测装置到一行中，比如WBC_before,WBC_3d,WBC_7d
temp_list=[]
for i in range(df_VPA_indicators.shape[0]):
    case_no=df_VPA_indicators.loc[i,'case_no']
    print(case_no)
    # drug_time
    drug_start_datetime=df_VPA_indicators.loc[i,'start_datetime']
    # 肝功数据
    temp_gan=df_test_gangong[df_test_gangong['case_no']==case_no]
    gan_before=temp_gan[temp_gan['test_date']<=drug_start_datetime]
    gan_after=temp_gan[temp_gan['test_date']>drug_start_datetime]
    if np.sum(gan_before['肝功_label']) >0:
        df_VPA_indicators.loc[i,'肝功_before']=1
    else:
        df_VPA_indicators.loc[i,'肝功_before']=0
        
    if np.sum(gan_after['肝功_label']) >0:
        df_VPA_indicators.loc[i,'肝功_after']=1
    else:
        df_VPA_indicators.loc[i,'肝功_after']=0
    # 皮疹数据
    temp_pizhen=df_test_pizhen[df_test_pizhen['case_no']==case_no]
    pizhen_before=temp_pizhen[temp_pizhen['start_datetime']<=drug_start_datetime]
    pizhen_after=temp_pizhen[temp_pizhen['start_datetime']>drug_start_datetime]
    if pizhen_before.shape[0]>0:
        df_VPA_indicators.loc[i,'皮疹_before']=1
    else:
        df_VPA_indicators.loc[i,'皮疹_before']=0
    if pizhen_after.shape[0]>0:
        df_VPA_indicators.loc[i,'皮疹_after']=1
    else:
        df_VPA_indicators.loc[i,'皮疹_after']=0
    # 恶心数据
    temp_exin=df_test_exin[df_test_exin['case_no']==case_no]
    exin_before=temp_exin[temp_exin['start_datetime']<=drug_start_datetime]
    exin_after=temp_exin[temp_exin['start_datetime']>drug_start_datetime]
    if exin_before.shape[0]>0:
        df_VPA_indicators.loc[i,'恶心_before']=1
    else:
        df_VPA_indicators.loc[i,'恶心_before']=0
    if exin_after.shape[0]>0:
        df_VPA_indicators.loc[i,'恶心_after']=1
    else:
        df_VPA_indicators.loc[i,'恶心_after']=0
    # 血小板计数
    temp_PLT=df_test_PLT[df_test_PLT['case_no']==case_no]
    PLT_before=temp_PLT[temp_PLT['test_date']<=drug_start_datetime]
    PLT_after=temp_PLT[temp_PLT['test_date']>drug_start_datetime]
    if np.sum(PLT_before['PLT_label']) >0:
        df_VPA_indicators.loc[i,'PLT_before']=1
    else:
        df_VPA_indicators.loc[i,'PLT_before']=0
        
    if np.sum(PLT_after['PLT_label']) >0:
        df_VPA_indicators.loc[i,'PLT_after']=1
    else:
        df_VPA_indicators.loc[i,'PLT_after']=0
    # 计算安全性指标
    if np.sum(df_VPA_indicators.loc[i,['肝功_before','皮疹_before','恶心_before','PLT_before']])>0:
        df_VPA_indicators.loc[i,'security_before']=1
    else:
        df_VPA_indicators.loc[i,'security_before']=0
    
    if np.sum(df_VPA_indicators.loc[i,['肝功_after','皮疹_after','恶心_after','PLT_after']])>0:
        df_VPA_indicators.loc[i,'security_after']=1        
    else:
        df_VPA_indicators.loc[i,'security_after']=0
    # 
    if (df_VPA_indicators.loc[i,'security_before']==0) & (df_VPA_indicators.loc[i,'security_after']==1):
        df_VPA_indicators.loc[i,'security_label']=1
    else:
        df_VPA_indicators.loc[i,'security_label']=0

4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
4922567
5386601
5386601
5386601
5386601
5386601
5386601
5386601
5386601
8981054
8981054
8981054
8981054
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
7251581
4045163
4045163
4045163
4045163
4045163
4045163
4045163
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
7661760
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
9123719
5407981
5407981
5407981
5407981
5407981
5407981
5407981
5407981
5407981
5407981
5407981
5407981
9818116
9818116
9818116
9818116
9818116
9818116
9818116
9818116
6435824
6435824
6435824
6435824
6435824
6435824
6435824
6435824
6435824
6435824


9910592
9910592
9910592
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
4821017
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
10001302
5302878
5302878
5302878
5302878
5302878
5302878
4283749
4283749
4283749
4283749
4283749
4283749
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
10019847
7830248
7830248
7830248
7830248
8542045
8542045
8542045
8542045
8542045
8542045
8542045
5612768
5612768
5612768
5612768
5612768
5612768
5612768
5612768
5612768
5612768
5612768
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556
7882556

7234799
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7284844
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7309258
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7937051
7423611
7423611
7423611
7423611
7423611
7423611
7423611
7423611
8368444
8368444
8368444
8368444
8368444
8368444
8368444
8368444
8368444
8368444


In [215]:
print(df_VPA_indicators.shape)
print(df_VPA_indicators['patient_id'].nunique())

(2610, 41)
145


In [216]:
df_VPA_indicators.head()

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc,...,肝功_after,皮疹_before,皮疹_after,恶心_before,恶心_after,PLT_before,PLT_after,security_before,security_after,security_label
0,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,1285315,4922567,丙戊酸钠缓释片(德巴金)[0.5g*30片],1,0.5g*30片,0.5,2,2018-11-10 15:39:01,2018-11-30 08:15:00,停止,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [217]:
df_VPA_indicators.to_excel(project_path+'/data/processed_data/df_6.2_安全性指标转置.xlsx')

# 添加丙戊酸TDM检测

## 提取tdm检测数据

In [218]:
# 提取用药后不良反应发生前的甲氨蝶呤血药浓度检测
df_test_VPA=df_test[df_test['project_name'].str.contains('丙戊酸|VPA')]
# 排序
df_test_VPA=df_test_VPA.sort_values(['patient_id','case_no','test_date'])
df_test_VPA=df_test_VPA.reset_index(drop=True)

In [219]:
df_test_VPA['tdm_date']=df_test_VPA['test_date'].apply(lambda x: str(x).split(' ')[0])

In [220]:
df_test_VPA.isnull().any()

test_record_id        False
patient_id            False
case_no                True
test_date             False
clinical_diagnosis     True
sample_type           False
sample_time            True
project_name          False
test_result           False
result_unit           False
is_normal             False
refer_scope            True
synonym               False
tdm_date              False
dtype: bool

In [221]:
df_test_VPA=df_test_VPA[df_test_VPA.case_no.notnull()]

In [222]:
print(df_test_VPA.shape)
print(df_test_VPA['patient_id'].nunique())

(349, 14)
218


In [223]:
# 提取患者的tdm检测
df_test_VPA=df_test_VPA[df_test_VPA['patient_id'].isin(df_VPA['patient_id'].unique())]
df_test_VPA=df_test_VPA.sort_values(['patient_id','test_date'])
df_test_VPA=df_test_VPA.reset_index(drop=True)

In [224]:
print(df_test_VPA.shape)
print(df_test_VPA['patient_id'].nunique())

(349, 14)
218


In [225]:
# 保存丙戊酸血药浓度检测
df_test_VPA.to_excel(project_path+'/data/processed_data/df_7.1_test_VPA.xlsx')

## 合并tdm检测数据

In [228]:
import datetime
# 合并甲氨蝶呤tdm检测信息
# 如果tdm用药前3天含有非稳态用药，删除；否则，保留。
temp_list=[]
for i in np.unique(df_VPA_indicators['case_no']):
    # 稳态 丙戊酸 temp
    temp_steady=df_VPA_indicators[df_VPA_indicators['case_no']==i]
    temp_steady=temp_steady.sort_values(['patient_id','case_no','分裂日期'])
    temp_steady=temp_steady.reset_index(drop=True)
    
    # 稳态前 丙戊酸 temp
    temp_before=df_VPA_modification[df_VPA_modification['case_no']==i]
    temp_before=temp_before.sort_values(['patient_id','case_no','分裂日期'])
    temp_before=temp_before.reset_index(drop=True)

    # tdm temp
    temp_tdm=df_test_VPA[df_test_VPA['case_no']==i]
    temp_tdm=temp_tdm.sort_values(['patient_id','case_no','test_date'])
    temp_tdm=temp_tdm.reset_index(drop=True)
    # 判断tdm前3天内是否有非稳态用药
    for j in range(temp_tdm.shape[0]):
        # 提取tdm检测日期
        tdm_date=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])
        # 提取稳态前tdm前3天用药
        temp_before_drug=temp_before[(pd.to_datetime(temp_before['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
                                    (pd.to_datetime(temp_before['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 提取稳态时tdm前3天用药
        temp_steady_drug=temp_steady[(pd.to_datetime(temp_steady['分裂日期'])<=tdm_date-datetime.timedelta(days=1))&
                            (pd.to_datetime(temp_steady['分裂日期'])>=tdm_date-datetime.timedelta(days=3))]
        # 如果稳态前和稳态时用药有变化，则舍弃tdm数据，否则，并入到VPA用药数据中
        if temp_before_drug.shape[0] != temp_steady_drug.shape[0]:
            continue
        else:
            if temp_steady_drug.shape[0]>0:
                # 获取最近用药的索引
                index=temp_steady_drug.index.tolist()[-1]
                temp_steady_drug.loc[index,'tdm_date']=temp_tdm.loc[j,'tdm_date']
                temp_steady_drug.loc[index,'test_result']=temp_tdm.loc[j,'test_result']
        temp_list.append(temp_steady_drug)

In [229]:
temp_tdm

,test_record_id,patient_id,case_no,test_date,clinical_diagnosis,sample_type,sample_time,project_name,test_result,result_unit,is_normal,refer_scope,synonym,tdm_date
0,800008575295||Y006||1,8374053,9963165,2020-08-14 11:18:00,双相情感障碍，目前为混合性发作,血浆,NaT,丙戊酸药物浓度监测,79.70,ug/ml,N,50.00-100.00,VPA,2020-08-14


In [230]:
df_VPA_tdm = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [231]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(419, 43)
110
110


In [232]:
df_VPA_tdm.columns

Index(['patient_id', 'case_no', 'drug_name', 'amount', 'drug_spec', 'dosage',
       'frequency', 'start_datetime', 'end_datetime', 'statusdesc', '日剂量',
       'start_date', 'end_date', 'long_d_order', '分裂日期', '起始标签', '时间标签',
       'gender', 'age', '身高', '体重', 'BMI', '高血压', '糖尿病', '脑器质疾病', '肝炎', '肾病',
       'adm_date', 'dis_date', '住院时长', '肝功_before', '肝功_after', '皮疹_before',
       '皮疹_after', '恶心_before', '恶心_after', 'PLT_before', 'PLT_after',
       'security_before', 'security_after', 'security_label', 'tdm_date',
       'test_result'],
      dtype='object')

In [233]:
# 保存添加tdm检测的甲氨蝶呤用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.2_添加tdm检测.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 删除tdm为空的数据

In [234]:
df_VPA_tdm=df_VPA_tdm[df_VPA_tdm['test_result'].notnull()].reset_index(drop=True)

In [235]:
print(df_VPA_tdm.shape)
print(df_VPA_tdm['patient_id'].nunique())
print(df_VPA_tdm['case_no'].nunique())

(142, 43)
110
110


In [236]:
# 保存tdm非空的用药
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_7.3_tdm非空用药.xlsx')
df_VPA_tdm.to_excel(writer)
writer.save()

## 插入有效性标签

In [238]:
df_VPA_tdm['efficiency_label']=df_VPA_tdm['test_result'].apply(lambda x: 1 if 50<=float(x)<=100 else 0)

In [239]:
df_VPA_tdm.to_excel(project_path+'/data/processed_data/df_7.4_插入有效性标签.xlsx')

## 统计用药和tdm先后

In [240]:
# 丙戊酸首次用药
# 排序
df_VPA=df_VPA.sort_values(['patient_id','case_no','start_datetime'])
# 取第一次用药
df_VPA_first=df_VPA.drop_duplicates(['case_no'],keep='first')

In [241]:
print(df_VPA_first.shape)
print(df_VPA_first['patient_id'].nunique())

(241, 10)
218


In [242]:
# tdm首次检测
# 排序
df_test_VPA=df_test_VPA.sort_values(['patient_id','case_no','test_date'])
# 取第一次tdm
df_test_VPA_first=df_test_VPA.drop_duplicates(['case_no'],keep='first')

In [243]:
print(df_test_VPA_first.shape)
print(df_test_VPA_first['patient_id'].nunique())

(241, 14)
218


In [244]:
# tdm时间早于用药的id
temp_list=[]
for i in df_VPA_first.case_no.unique():
    temp_VPA=df_VPA_first[df_VPA_first['case_no']==i]
    temp_VPA=temp_VPA.reset_index(drop=True)
    temp_tdm=df_test_VPA_first[df_test_VPA_first['case_no']==i]
    temp_tdm=temp_tdm.reset_index(drop=True)
    if temp_VPA.loc[0,'start_datetime'] >= temp_tdm.loc[0,'test_date']:
        temp_list.append(temp_VPA)

In [245]:
df_temp_compare = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

In [246]:
print(df_temp_compare.shape)
print(df_temp_compare['patient_id'].nunique())

(2, 10)
2


In [247]:
df_temp_compare.to_excel(project_path+'/data/processed_data/df_7.4_比较tdm和用药时间先后.xlsx')

## 统计tdm检测前用药天数

In [248]:
# 纳排后不同用药稳态在tdm检测前天数

In [249]:
# 1天用药稳态
df_VPA_steady_1d=df_VPA_modification.copy()

In [250]:
# 2天用药稳态
df_VPA_steady_2d=df_VPA_modification.copy()
# 到稳态血药浓度（同一剂量至少连续服用3天)
temp_list=[]
for i in df_VPA_steady_2d['case_no'].unique():
    print(i)
    temp=df_VPA_steady_2d[df_VPA_steady_2d['case_no']==i]
    temp=temp.reset_index(drop=True)
    j=0
    while j < temp.shape[0]:
#         print(j)
        # 连续服药3天，tdm达到稳态。1.剂量连续3天；2. 时间连续3天
        dosage_judge=temp.loc[j,'日剂量']    
        # 1. 剂量判断
        break_bool=False
        for k in range(j+1,temp.shape[0]):
            differ = (pd.to_datetime(temp.loc[k,'分裂日期'])-pd.to_datetime(temp.loc[k-1,'分裂日期'])).days
            if dosage_judge != temp.loc[k,'日剂量'] or differ != 1:
                # 判断是否中断
                break_bool=True
                break
        # 是否是最后一天
        if k != temp.shape[0]-1:
            # 是否是稳态
            if k-1-j>=1:
                temp_list.append(temp.loc[j:k-1,])
            j=k
            continue
        else:
            # 最后一天是否中断
            if break_bool:
                # 是否是稳态
                if k-1-j>=1:
                    temp_list.append(temp.loc[j:k-1,])
            else:
                if k-j >=1:
                    temp_list.append(temp.loc[j:k,])
            j=k+1
# list转换为DataFrame
df_VPA_steady_2d = pd.concat(temp_list,axis=0).reset_index(drop=True)
del temp_list

10001302
10005574
10019847
10024628
10028911
10130473
10146112
10171606
10200584
10293806
10297961
10338392
10343367
10357709
10368285
10389652
10526969
10559425
10582637
10588084
10616235
10674663
10692337
10830666
10859517
10864711
10877290
10914758
10941636
10989230
11022153
11069250
11103754
11200975
11223919
11238946
11291598
11351405
11547061
11747590
11787937
12001085
12055514
12069121
12070485
12165203
12210690
12238352
12297734
12313528
12325442
12327058
12415154
12550325
12610841
12631919
12693940
12696147
12782405
12835527
12855725
13022476
13063464
13150462
13176502
13266036
13317922
13335769
13397293
13425571
13491055
13561501
13692578
13817948
13951019
13997754
14040566
14103871
14107745
14120909
14135942
14184132
14322457
14400275
14701280
14712892
14900947
15026052
4028318
4045163
4069083
4103620
4112018
4133350
4151975
4191397
4216082
4236927
4255479
4267891
4283749
4304073
4364618
4366097
4446056
4446680
4536465
4553217
4560293
4569640
4579522
4592169
4707313
4761372


In [251]:
print(df_VPA_steady_2d.shape)
print(df_VPA_steady_2d['patient_id'].nunique())

(2417, 17)
156


In [252]:
df_VPA_steady_2d.to_excel(project_path+'/data/df_VPA_steady_2d.xlsx')

In [253]:
df_aaa=pd.DataFrame(data=[],columns=['aaa','bbb'])
print(df_aaa.shape[0])

0


In [254]:
# 3天用药稳态
drug_days_list=[]
case_no_list=[]
for i in df_VPA_steady_1d.case_no.unique():
#     print(i)
    case_no_list.append(i)
    # 1天稳态
    temp_VPA_1d=df_VPA_steady_1d[df_VPA_steady_1d['case_no']==i]
    temp_VPA_1d=temp_VPA_1d.reset_index(drop=True)
    # 2天稳态
    temp_VPA_2d=df_VPA_steady_2d[df_VPA_steady_2d['case_no']==i]
    temp_VPA_2d=temp_VPA_2d.reset_index(drop=True)
    # 3天稳态
    temp_VPA=df_VPA_steady[df_VPA_steady['case_no']==i]
    temp_VPA=temp_VPA.reset_index(drop=True)
    # tdm检测
    temp_tdm=df_test_VPA[df_test_VPA['case_no']==i]
    temp_tdm=temp_tdm.reset_index(drop=True)
    day_list=[]
    for j in range(temp_tdm.shape[0]):
        # 1天稳态
        temp2=temp_VPA_1d[(pd.to_datetime(temp_VPA_1d['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=3))&
                          (temp_VPA_1d['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<1:
            day_list.append(0)
        else:
            day_list.append(1)
        # 2天稳态
        temp2=temp_VPA_2d[(pd.to_datetime(temp_VPA_2d['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=2))&
                          (temp_VPA_2d['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<2:
            day_list.append(0)
        else:
            day_list.append(1)
        # 3天稳态
        temp2=temp_VPA[(pd.to_datetime(temp_VPA['分裂日期'])>=pd.to_datetime(temp_tdm.loc[j,'tdm_date'])-datetime.timedelta(days=3))&
                      (temp_VPA['分裂日期']<temp_tdm.loc[j,'tdm_date'])]
        if temp2.shape[0]<3:
            day_list.append(0)
        else:
            day_list.append(1)
    drug_days_list.append(day_list)

In [255]:
df_tdm_days_count=pd.DataFrame(data=drug_days_list,index=case_no_list,columns=['1d_tdm1','2d_tdm1','3d_tdm1',
                                                                               '1d_tdm2','2d_tdm2','3d_tdm2',
                                                                               '1d_tdm3','2d_tdm3','3d_tdm3',
                                                                               '1d_tdm4','2d_tdm4','3d_tdm4',
                                                                               '1d_tdm5','2d_tdm5','3d_tdm5'])

In [256]:
df_tdm_days_count.to_excel(project_path+'/data/processed_data/df_7.5_统计tdm检测前用药天数.xlsx')

# 增加联合用药

## 提取联合用药

In [257]:
# 提取患者的联合用药
df_drug_other=df_doctor_order[~ df_doctor_order['drug_name'].str.contains('丙戊酸|葡萄糖|氯化钠')]
# 患者限定
df_drug_other=df_drug_other[df_drug_other['case_no'].isin(df_VPA_tdm['case_no'].unique())]
df_drug_other=df_drug_other.reset_index(drop=True)

In [258]:
print(df_drug_other.shape)
print(df_drug_other['patient_id'].nunique())

(10749, 10)
110


In [259]:
df_drug_other

,patient_id,case_no,drug_name,amount,drug_spec,dosage,frequency,start_datetime,end_datetime,statusdesc
0,6873870,10001302,齐拉西酮冻干粉针(江都医院)[30mg*1瓶],1.0,30mg*1瓶,20mg,ONCE,2020-08-15 18:59:57,NaT,核实
1,6873870,10001302,行为矫正治疗,20.0,None,None,1/日,2020-08-15 19:45:12,2020-09-04 11:38:00,停止
2,6873870,10001302,静脉输液第二组起每组,3.0,None,None,1/日,2020-08-17 11:24:51,2020-08-20 10:11:04,停止
3,6873870,10001302,脑苷肌肽注射液(欧迪美)[2ml*1支],6.0,2ml*1支,4ml,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
4,6873870,10001302,静脉输液第二组起每组,3.0,None,None,1/日,2020-08-17 11:25:27,2020-08-20 10:11:09,停止
...,...,...,...,...,...,...,...,...,...,...
10744,8334957,9890262,暗示治疗,14.0,None,None,1/日,2020-08-04 12:52:35,2020-08-18 08:13:00,停止
10745,8334957,9890262,心理治疗(中级及以下职称),1.0,None,None,ONCE,2020-08-11 17:11:41,NaT,核实
10746,8334957,9890262,行为观察和治疗,14.0,None,None,1/日,2020-08-04 12:52:35,2020-08-18 08:13:00,停止
10747,8334957,9890262,舍曲林片(左洛复)[50mg*14片],1.0,50mg*14片,50mg,1/日,2020-08-17 11:19:21,NaT,核实


In [260]:
df_drug_other.to_excel(project_path+'/data/processed_data/df_8.1_drug_other.xlsx')

## 统计联合用药频数

In [262]:
# 统计用药频数，由业务人员确定选用哪些联合用药。
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_VPA_drug_other=df_VPA_tdm.copy()
temp_list=[]
for i in range(df_VPA_drug_other.shape[0]):
    case_no=df_VPA_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 时间限定：用药前3天-tdm检测前
    drug_time=df_VPA_drug_other.loc[i,'start_date']
    tdm_time=df_VPA_drug_other.loc[i,'tdm_date']
    # 提取用药前3天内-tdm前的联合用药。
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(drug_time)-datetime.timedelta(days=3)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    temp_list.append(drug_other_1)

df_drug_other_choose = pd.concat(temp_list,axis=0)
del temp_list
df_drug_other_choose=df_drug_other_choose.drop_duplicates().reset_index(drop=True)
drug_name_counts=df_drug_other_choose.drug_name.value_counts()
df_drug_name_counts=pd.DataFrame(data={'药物名称':drug_name_counts.index,
                                     '频次':drug_name_counts.values})
print(df_drug_name_counts.shape)
# 保存联合用药名
df_drug_name_counts.to_excel(project_path+'/data/processed_data/df_8.2_统计联合用药频数.xlsx')

(149, 2)


## 合并联合用药

In [263]:
# 业务人员给定的联合用药
emotion_wending=['拉莫三嗪', '氯硝西泮', '地西泮', '氢溴酸伏硫西汀片', '阿戈美拉汀', '西酞普兰']
anti_psychotic=['奥氮平', '喹硫平', '阿立哌唑', '齐拉西酮', '利培酮', '帕利哌酮', '氯氮平', '氯丙嗪']
anti_anxiety=['劳拉西泮', '阿普唑仑', '坦度螺酮', '舍曲林片', '艾司西酞普兰', '氟西汀', '帕罗西汀', '米氮平', '曲唑酮']
sleeping_pills=['右佐匹克隆', '唑吡坦', '舒眠胶囊', '乌灵胶囊', '艾司唑仑']
behavior_therapy=['暗示治疗', '引导式教育训练', '行为矫正治疗', '冲动行为干预治疗']

In [264]:
df_emotion_wending=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('拉莫三嗪|氯硝西泮|地西泮|氢溴酸伏硫西汀片|阿戈美拉汀|西酞普兰')]
df_emotion_wending.to_excel(project_path+'/data/processed_data/df_emotion_wending.xlsx')

In [265]:
df_anti_psychotic=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('奥氮平|喹硫平|阿立哌唑|齐拉西酮|利培酮|帕利哌酮|氯氮平|氯丙嗪')]
df_anti_psychotic.to_excel(project_path+'/data/processed_data/df_anti_psychotic.xlsx')

In [266]:
df_sleeping_pills=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('右佐匹克隆|唑吡坦|舒眠胶囊|乌灵胶囊|艾司唑仑')]
df_sleeping_pills.to_excel(project_path+'/data/processed_data/df_sleeping_pills.xlsx')

In [267]:
df_behavior_therapy=df_drug_other_choose[df_drug_other_choose.drug_name.str.contains('暗示治疗|引导式教育训练|行为矫正治疗|冲动行为干预治疗')]
df_behavior_therapy.to_excel(project_path+'/data/processed_data/df_behavior_therapy.xlsx')

In [270]:
# 联合用药时间限定：结束时间>=用药前3天内+开始时间<=tdm检测前，取交集
df_VPA_drug_other=df_VPA_tdm.copy()
df_VPA_drug_other=df_VPA_drug_other.reset_index(drop=True)
temp_list=[]
for i in range(df_VPA_drug_other.shape[0]):
    case_no=df_VPA_drug_other.loc[i,'case_no']
    # 联合用药
    drug_other=df_drug_other[df_drug_other['case_no']==case_no]
    drug_other=drug_other.reset_index(drop=True)
    # 分裂联合用药日期
    drug_other['start_date']=drug_other['start_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    drug_other['end_date']=drug_other['end_datetime'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) 
    # 结束时间>=用药前3天内+开始时间<=不良反应前，取交集
    drug_time=df_VPA_drug_other.loc[i,'start_date']
    tdm_time=df_VPA_drug_other.loc[i,'tdm_date']
    drug_other_1=drug_other[(drug_other['end_date']>=pd.to_datetime(drug_time)-datetime.timedelta(days=3)) &
                         (drug_other['start_date']<=pd.to_datetime(tdm_time))]
    # 联合用药列表
    drug_other_list=drug_other_1.drug_name.unique()
    # 联合用药转置
    selet_drug_other=['情感稳定剂','抗精神病药','抗焦虑药','安眠药','行为干预治疗']
    drug_other_name_list=[emotion_wending,anti_psychotic,anti_anxiety,sleeping_pills,behavior_therapy]
    for name,name_list in zip(selet_drug_other,drug_other_name_list):
        match_value=[1 if re.match(x,y) else 0 for x in name_list for y in drug_other_list]
        if np.sum(match_value)>0:
            df_VPA_drug_other.loc[i,name]=1
        else:
            df_VPA_drug_other.loc[i,name]=0

In [271]:
print(df_VPA_drug_other.shape)
print(df_VPA_drug_other['patient_id'].nunique())

(142, 49)
110


In [273]:
# 时间限定的联合用药
df_VPA_drug_other.to_excel(project_path+'/data/processed_data/df_8.3_合并联合用药.xlsx')

# 合并其他相关检测

## 提取其他检测

In [274]:
# 提取甲氨蝶呤患者的所有检测
df_test_other=df_test[df_test['case_no'].isin(df_VPA_tdm['case_no'].unique())]

In [275]:
# 提取患者的其他检测
df_test_other = df_test_other[~ df_test_other['project_name'].str.contains('丙戊酸')]
df_test_other = df_test_other.reset_index(drop=True)

In [276]:
df_test_other.isnull().any()

test_record_id        False
patient_id            False
case_no               False
test_date             False
clinical_diagnosis    False
sample_type           False
sample_time            True
project_name          False
test_result           False
result_unit            True
is_normal              True
refer_scope            True
synonym                True
dtype: bool

In [277]:
print(df_test_other.shape)
print(df_test_other['patient_id'].nunique())

(16335, 13)
110


In [278]:
# 保存其他检测
df_test_other.to_csv(project_path+'/data/processed_data/df_9.1_test_other.csv')

## 统计其他检测项频数

In [279]:
df_test_other_name=pd.DataFrame(df_test_other['project_name'].value_counts())

In [280]:
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.2_统计其他检测项频数.xlsx')
df_test_other_name.to_excel(writer)
writer.save()

## 合并其他检测

In [281]:
def judge_float(x):
    try:
        a=float(x)
        return a
    except:
        return np.NaN

In [285]:
df_VPA_test_other=df_VPA_drug_other.copy()
# 将用药后不良反应发生前的最近一次其他检测转置到一行中
for i in range(df_VPA_test_other.shape[0]):
    case_no=df_VPA_test_other.loc[i,'case_no']
    tdm_date=df_VPA_test_other.loc[i,'tdm_date']
    # case_no的其他检测
    test_other= df_test_other[df_test_other['case_no']==case_no]
    test_other=test_other.reset_index(drop=True)
    test_other['test_date']=test_other['test_date'].apply(lambda x: pd.to_datetime(str(x).split(' ')[0])) # 分裂检测日期
    # 取用药后不良反应发生前的其他检测，此案例中start_datetime>tdm_date-1。
    test_other_1=test_other[(test_other['test_date']>=pd.to_datetime(tdm_date)-datetime.timedelta(days=3))&
                            (test_other['test_date']<=tdm_date)]
    # 取最近的一次其他检测
    test_other_1=test_other_1.sort_values(['test_date'])
    test_other_1=test_other_1.drop_duplicates(['project_name'],keep='last')
    test_other_1=test_other_1.reset_index(drop=True)
    
    # 检验项转置为表头
    if test_other_1.shape[0]>0:
        test_other_1['test_result']=test_other_1['test_result'].apply(judge_float)
        test_other_pivot=test_other_1.pivot_table('test_result',['case_no'], 'project_name')
        test_other_pivot=test_other_pivot.reset_index(drop=True)
        for j in test_other_pivot.columns[1:]:
            df_VPA_test_other.loc[i,j]=test_other_pivot.loc[0,j]
#     break
#     if case_no == '410308':
#         print(temp.loc[0,'start_datetime'])
#         print(temp.loc[0,'bmd_time'])
#         print(test_other_1)
#         print(test_other_2[test_other_2['project_name']=='肌酐(干式)'])
#         break

In [286]:
print(df_VPA_test_other.shape)
print(df_VPA_test_other['patient_id'].nunique())

(142, 172)
110


In [287]:
# 合并其他检测
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.3_合并其他检测.xlsx')
df_VPA_test_other.to_excel(writer)
writer.save()

## 删除缺失率超过50%的列

In [288]:
# 删除缺失超过50%的检测
for i in df_VPA_test_other.columns:
    x=df_VPA_test_other[i].isnull().sum()
    per_x = x/df_VPA_tdm.shape[0]  
    if per_x >= 0.5:
        print(i)
        del df_VPA_test_other[i]

天门冬氨酸氨基转移酶
总胆红素
总蛋白
球蛋白
白/球比值
白蛋白
直接胆红素
转氨酶比值
间接胆红素
催乳素
卵泡刺激素
可溶性转铁蛋白受体
孕酮
总铁结合力
睾酮
转铁蛋白
转铁蛋白饱和度
铁离子
铁蛋白
雌二醇
黄体生成素
不饱和铁结合力
丙型肝炎病毒抗体[电化学发光]
丙氨酸氨基转移酶
中性粒细胞总数
中性粒细胞百分数
乙型肝炎e抗体测定
乙型肝炎e抗原测定
乙型肝炎核心抗体测定
乙型肝炎表面抗体测定
乙型肝炎表面抗原测定
低密度脂蛋白胆固醇
促甲状腺素
凝血酶原时间比率
凝血酶原活动度
凝血酶时间测定
单核细胞总数
单核细胞百分数
嗜碱性粒细胞总数
嗜碱性粒细胞百分数
嗜酸性粒细胞总数
嗜酸性粒细胞百分数
国际标准比率
大型血小板比值
小而密低密度脂蛋白胆固醇
尿比重(拆射计法)
尿素
尿酸
平均RBC血红蛋白浓度
平均红细胞体积
平均红细胞血红蛋白量
平均血小板容积
心型肌酸激酶
总二氧化碳
总胆固醇
总钙
无机磷
极低密度脂蛋白胆固醇
梅毒螺旋体特异性抗体[电化学发光法]
氯离子
活化部分凝血活酶时间
淋巴细胞总数
淋巴细胞百分数
游离T3
游离T4
甘油三酯
甲状三碘原氨酸
甲状腺素
白介素-6
白细胞计数
红细胞分布宽度-CV
红细胞分布宽度-SD
红细胞比积测定
红细胞计数
肌酐
肌酸激酶
胱抑素-C
脂蛋白a
葡萄糖
血小板分布宽度
血小板比积测定
血小板计数
血浆D-二聚体测定
血浆凝血酶原时间测定
血浆纤维蛋白原测定
血清载脂蛋白E测定
血红蛋白测定
载脂蛋白A
载脂蛋白B
载脂蛋白比值
酸碱浓度
钠离子
钾离子
镁离子
高密度脂蛋白胆固醇
乳酸脱氢酶
皮质醇
糖化血红蛋白
非高密度脂蛋白胆固醇
脂肪酶
a-羟丁酸脱氢酶
r-谷氨酰基转移酶
前白蛋白
同型半胱氨酸
尿上皮细胞(仪器定量)
尿白细胞(仪器定量)
尿管型(仪器定量)
尿红细胞(仪器定量)
尿细菌(仪器定量)
尿结晶
尿酵母菌
岩藻糖苷酶
总胆汁酸
碱性磷酸酶
红细胞沉降率测定
胆碱脂酶
尿沉渣镜检白细胞
抗甲状腺球蛋白抗体
抗甲状腺过氧化物酶
降钙素原
尿沉渣镜检红细胞
促甲状腺受体自身抗体
镜检红细胞


In [289]:
print(df_VPA_test_other.shape)
print(df_VPA_test_other['patient_id'].nunique())

(142, 49)
110


In [290]:
# 保存删除缺失率后的数据
writer=pd.ExcelWriter(project_path+'/data/processed_data/df_9.4_删除缺失超过50%的列.xlsx')
df_VPA_test_other.to_excel(writer)
writer.save()

## 人工删除没有意义的指标